In [2]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zyth0001\AppData\Roaming\nltk_data...


True

In [1]:
from polyjuice import Polyjuice

c:\Users\zyth0001\Anaconda3\envs\dialog-classification\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from polyjuice import Polyjuice
pj = Polyjuice(model_path="uw-hai/polyjuice")
text = "It is great for kids."
perturbations = pj.perturb(text)

c:\Users\zyth0001\Anaconda3\envs\dialog-classification\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.


s: {'generated_token_ids': [1026, 318, 1049, 329, 3988, 13, 1279, 91, 11766, 5945, 91, 29, 685, 2588, 605, 60, 685, 9148, 15154, 60, 632, 318, 1049, 329, 3988, 13, 685, 5188, 47, 60, 38144, 9936, 685, 15037, 45532, 60, 220, 50256]}


AttributeError: 'list' object has no attribute 'split'

In [ ]:
from transformers import AutoTokenizer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import random

model_path = "uw-hai/polyjuice"
generator = pipeline("text-generation", 
    model=AutoModelForCausalLM.from_pretrained(model_path), 
    tokenizer=AutoTokenizer.from_pretrained(model_path),
    framework="pt", device=-1)

In [35]:
text = "I like painting with oil on a canvas."
#text = "What is the first law of thermodynamics?"

operations = ['negation', 'quantifier', 'shuffle', 'lexical', 'resemantic', 'insert', 'delete', 'restructure']
blank_operations = [ 'shuffle', 'lexical', 'resemantic', 'insert', 'delete', 'restructure']

generated_sentences = []

for operation in operations:
    prompt_text =  text + " <|perturb|> ["+ operation +"]"
    generated_sentences.append(generator(prompt_text, num_beams=3, num_return_sequences=1))

# todo randomly set [BLANK]
text_parts = text.split()
for operation in blank_operations:
    replace_token = random.choice(text_parts)
    randomized_blank = text.replace(replace_token, "[BLANK]")
    prompt_text = text + " <|perturb|> ["+ operation +"] " + randomized_blank
    generated_sentences.append(generator(prompt_text, num_beams=3, num_return_sequences=1))

sentences = []
for elements_list in generated_sentences:
    for element in elements_list:
        #print(element)
        sentence = element['generated_text']
        sentence_parts = sentence.split(' <|perturb|> ')
        perturbation = sentence_parts[1]
        for operation in operations:
            perturbation = perturbation.replace("["+operation+"]","")
        perturbation_parts = perturbation.split(" [SEP] ")
        perturbed_sentence = perturbation_parts[0].replace("[BLANK]", "{}")
        answer_sentence = perturbation_parts[1]
        answers = answer_sentence.split(" [ANSWER] ")
        sentence = perturbed_sentence.format(*answers)
        sentence = sentence.replace(' EMPTY ', " ").strip()
        if sentence not in sentences:
            sentences.append(sentence)

for sentence in sentences:
    print(sentence)



c:\Users\zyth0001\Anaconda3\envs\dialog-classification\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


I don't like painting with oil on a canvas.
I like painting with oil on a canvas.
I like painting with wood on canvas.
I like painting with wood on a canvas.
I like painting paintings with wood on a canvas.
I like painting on a canvas.
I prefer painting with oil on a canvas.
I like painting with wood oil on a canvas.
I like painting with wood instead of oil on a canvas.
I like painting with oil on a rainy day.
